In [1]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

from allennlp.predictors.predictor import Predictor
import spacy

PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )

sys.path.append( os.path.join( PATH ) )

with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)  
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/test.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem  )


In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

print( f"loading AllenNLP predictor from {BERT_PATH}" )
bert_model = Predictor.from_path( BERT_PATH )

print( f"loading spacy model from {SPACY_PATH}" )
nlp=spacy.load( SPACY_PATH )

transformer=ListTransformer( cas )

transformer.add_list_view( OldSofaID='html2textView', NewSofaID = 'ListView' )

reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas(  TEMPLATE_PATH , ROSofaID='ReportingObligationsView' )

reporting_obligations_finder.print_to_html( TEMPLATE_PATH, OUTPUT_PATH )


loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/test_test.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template
